# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686553


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:44,  1.60s/it]

Rendering models:  10%|█         | 3/30 [00:03<00:32,  1.19s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:37,  1.42s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:27,  1.08s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.22it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  3.94it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  3.75it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  3.67it/s]

not-logged-in-b1038df97c93b63613cc     0.001133
not-logged-in-3e461b5a5635ce55ae0f     0.001721
not-logged-in-20a275b53abec8db9f41     0.013049
not-logged-in-bc5fdb9ea8d611e60b8b     0.000298
not-logged-in-b6715ce74c77332a7fa3     0.011282
not-logged-in-48a0e5e746c7954388d8     0.421071
Linda_J._Berkel                        0.001263
ellianderjoy                           0.000450
yooman12                               0.009498
dejansson                              0.000498
earthling001                           0.000418
Marra                                  0.001581
clars915                               0.000939
not-logged-in-5f30b2bb9b7f9d49da41     0.003694
Brofessoramundsen                      0.013537
RaulE                                  0.000746
not-logged-in-9f30f411df9d28d2c541     0.000559
acapirala                              0.000417
helios1235                             0.000352
Lavadude                              15.727382
not-logged-in-15498ebaa693dd5a198d     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())